In [1]:
import torch
import torch.nn as nn
torch.cuda.is_available()
import pandas as pd

In [2]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.constants import POSITION_COLS, MASS_CLASS_COLS, BASIC_TRAINING_COLS, FORCE_CLASS_COLS, PUCK_SQUARE_DISTANCES, PUCK_ANGLE_FEATURES
from isaac.sanity import class_proportions
from isaac.models import initialise_model
from isaac.training import training_loop
import matplotlib.pyplot as plt
import numpy as np
import matplotlib2tikz

In [3]:
NORMALISE_DATA = True
BATCH_SIZE = 128
EPOCHS = 25
STEP_SIZE = 1
SEQ_END = 1800

In [4]:
TR_COLS = BASIC_TRAINING_COLS

In [5]:
train_trials = read_dataset("data/train_passive_trials.h5", n_trials=3500, cols=TR_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", n_trials=500, cols=TR_COLS)

100%|██████████| 500/500 [00:04<00:00, 117.08it/s]


# MASS TRAINING

In [6]:
INPUT_DIM = len(TR_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5
network_params = (INPUT_DIM, HIDDEN_DIM, N_LAYERS, OUTPUT_DIM, DROPOUT)

In [7]:
labels = ["1800", "1500", "1200", "900", "600", "300"]
stats_dfs = []

loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=MASS_CLASS_COLS, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA)

for seq_length, label in zip([1800, 1500, 1200, 900, 600, 300], labels):
    seq_start = SEQ_END - seq_length
    
    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["seq_length", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_start=seq_start, 
                                                                   seq_end=SEQ_END, step_size=STEP_SIZE)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["seq_length"] = label
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf("sequence_length_plots/mass_stats.h5", key="stats")

100%|██████████| 500/500 [00:00<00:00, 996.24it/s] 
Train_loss (0.90)	 Train_acc (58.17)	 Val_acc (51.40): 100%|██████████| 25/25 [08:37<00:00, 20.69s/it]
Train_loss (0.67)	 Train_acc (71.77)	 Val_acc (54.60): 100%|██████████| 25/25 [08:33<00:00, 20.51s/it]
Train_loss (0.82)	 Train_acc (62.34)	 Val_acc (56.00): 100%|██████████| 25/25 [08:42<00:00, 20.82s/it]
Train_loss (0.85)	 Train_acc (60.66)	 Val_acc (54.60): 100%|██████████| 25/25 [07:17<00:00, 17.59s/it]
Train_loss (0.82)	 Train_acc (62.00)	 Val_acc (49.40): 100%|██████████| 25/25 [07:18<00:00, 17.56s/it]
Train_loss (1.00)	 Train_acc (49.77)	 Val_acc (46.80): 100%|██████████| 25/25 [07:19<00:00, 17.57s/it]
Train_loss (1.00)	 Train_acc (48.51)	 Val_acc (44.80): 100%|██████████| 25/25 [05:52<00:00, 14.10s/it]
Train_loss (1.01)	 Train_acc (47.94)	 Val_acc (39.40): 100%|██████████| 25/25 [05:51<00:00, 14.01s/it]
Train_loss (0.91)	 Train_acc (57.09)	 Val_acc (49.20): 100%|██████████| 25/25 [05:49<00:00, 14.08s/it]
Train_loss (1.08)	 Tr

# FORCE TRAINING

In [8]:
labels = ["1800", "1500", "1200", "900", "600", "300"]
stats_dfs = []

loaders, scaler = prepare_dataset([train_trials, val_trials], class_columns=FORCE_CLASS_COLS, 
                                  training_columns=TR_COLS, batch_size=BATCH_SIZE, 
                                  normalise_data=NORMALISE_DATA)

for seq_length, label in zip([1800, 1500, 1200, 900, 600, 300], labels):
    seq_start = SEQ_END - seq_length
    
    for seed in [0, 42, 72]:
        df = pd.DataFrame(columns=["seq_length", "Epoch", "Loss"])


        model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed)
        epoch_losses, epoch_accuracies, best_model = training_loop(model, optimizer, error, loaders[0], 
                                                                   loaders[1], EPOCHS, seq_start=seq_start, 
                                                                   seq_end=SEQ_END, step_size=STEP_SIZE)
                
        df["Epoch"] = np.arange(EPOCHS)
        df["Loss"] = epoch_losses
        df["Train Accuracy"] = epoch_accuracies[0]
        df["Val Accuracy"] = epoch_accuracies[1]
        df["seq_length"] = label
        stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf("sequence_length_plots/force_stats.h5", key="stats")

100%|██████████| 500/500 [00:00<00:00, 1130.22it/s]
Train_loss (0.61)	 Train_acc (63.71)	 Val_acc (56.60): 100%|██████████| 25/25 [08:42<00:00, 20.89s/it]
Train_loss (0.76)	 Train_acc (57.71)	 Val_acc (50.80): 100%|██████████| 25/25 [08:41<00:00, 20.76s/it]
Train_loss (0.61)	 Train_acc (61.17)	 Val_acc (54.80): 100%|██████████| 25/25 [08:35<00:00, 20.63s/it]
Train_loss (0.60)	 Train_acc (67.71)	 Val_acc (58.00): 100%|██████████| 25/25 [07:12<00:00, 17.32s/it]
Train_loss (0.67)	 Train_acc (57.74)	 Val_acc (51.40): 100%|██████████| 25/25 [07:15<00:00, 17.62s/it]
Train_loss (0.58)	 Train_acc (59.66)	 Val_acc (51.00): 100%|██████████| 25/25 [07:21<00:00, 17.61s/it]
Train_loss (0.67)	 Train_acc (67.14)	 Val_acc (52.20): 100%|██████████| 25/25 [05:55<00:00, 14.26s/it]
Train_loss (0.68)	 Train_acc (66.51)	 Val_acc (54.20): 100%|██████████| 25/25 [05:55<00:00, 14.29s/it]
Train_loss (0.63)	 Train_acc (69.11)	 Val_acc (57.60): 100%|██████████| 25/25 [05:57<00:00, 14.25s/it]
Train_loss (0.68)	 Tr